In [1]:
from Game.GameLogic import Board
from random import choice as randchoice
import numpy as np
from copy import deepcopy as copy
from tqdm import tqdm_notebook as tqdm
import pickle,math
import time

In [2]:
def one_search(board,color):
    b=copy(board)
    current_color=copy(color)
    while not b.finished:
        move=randchoice(b.get_legal_moves(current_color))
    #         print(move+1)
        b.execute_move(move,current_color)
        current_color*=-1
    return b.winner

def search_one_move(board,move,color,times):
    b=copy(board)
    current_color=copy(color)
    b.execute_move(move,current_color)
    return sum([one_search(b,-current_color) for i in range(times)])

def search_next(board,color,times):
    time_start=time.time()
    moves=board.get_legal_moves(color)
    times_each=times//len(moves)
    print('search {} times for each move.'.format(times_each))
    move_value=[None]*len(moves)
    for i in tqdm(range(len(moves))):
        move_value[i]=search_one_move(board,moves[i],color,times_each)
    time_end=time.time()
    print('time cost',time_end-time_start,'s')
    return np.array(move_value)

In [213]:
def search(board,color):
    """
    This function performs one iteration of MCTS. It is recursively called
    till a leaf node is found. The action chosen at each node is one that
    has the maximum upper confidence bound as in the paper.

    Once a leaf node is found, the neural network is called to return an
    initial policy P and a value v for the state. This value is propogated
    up the search path. In case the leaf node is a terminal state, the
    outcome is propogated up the search path. The values of Ns, Nsa, Qsa are
    updated.

    NOTE: the return values are the negative of the value of the current
    state. This is done since v is in [-1,1] and if v is the value of a
    state for the current player, then its value is -v for the other player.

    Returns:
        v: the negative of the value of the current canonicalBoard / the previous move
    """

    s = board.tostring()
    times=10
    # terminal node : return the value of the game
    if board.finished:
        return -board.winner*color # the negative of the value of the previous move

    # leaf node : return the value given by the nnet
    if s not in Qs:
#         rand = one_search(board,color) # color : the color of the next move

        rand = sum([one_search(board,color) for i in range(times)])
        v = rand*color # the value of the next color
#         print('search result : {}, color : {}'.format(rand,color))
        Qs[s] = -v # Qs : the value of the previous move
        Ns[s] = times
        return -v

    # non-leaf node
    cur_best = -float('inf')
    best_move = []

    # pick the action with the highest upper confidence bound
#     print('legal moves : {}'.format(board.get_legal_moves(color)))
    for move in board.get_legal_moves(color):
#         print(move)
        smove=copy(board).execute_move(move,color).tostring()
        if smove in Ns:
            u = Qs[smove]/Ns[smove] + cpuct*math.sqrt(2*math.log(Ns[s])/Ns[smove])
        else:
            best_move=move
            break
            
        if u > cur_best:
            cur_best = u
            best_move = move
#         print('current move : {}, u : {}, best_move : {}'.format(move,u,best_move))
    if len(best_move)!=0:
        board_next=copy(board).execute_move(best_move,color)
    else:
        raise Exception('no best action found!')
#     print('best_move : {} , current color : {}'.format(best_move,color))

    # next recursion
    v = search(board_next,-color) # the value of the current color
    
    # back propagation for non-leaf node
    Qs[s]-=v # Qs : the value of the previous move
    Ns[s]+=times
    return -v


In [214]:
b=Board(9)
current_color=1
Qs={}
Ns={}
cpuct=1

In [302]:
for i in tqdm(range(40)):
    for j in range(100):
        search(b,current_color)

KeyboardInterrupt: 

In [299]:
Ns_list=[]
Qs_list=[]
for move in b.get_legal_moves(current_color):
    s = copy(b).execute_move(move,current_color).tostring()
    Ns_list.append(Ns[s])
    Qs_list.append(Qs[s])
max_move=np.array(Ns_list).argmax()
next_move=b.get_legal_moves(current_color)[max_move]
print(next_move+1,Ns_list[max_move],Qs_list[max_move])
print(np.array([Ns_list,Qs_list]))

[7 5] 3218 -767
[[ 347 3218  721  598]
 [-137 -767 -236 -205]]


In [300]:
b.execute_move(next_move,current_color)
current_color*=-1

In [301]:
b.execute_move(np.array([2,2])-1,current_color)
current_color*=-1

In [289]:
b.pieces

array([[-1,  0, -1,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0, -1, -1, -1,  0, -1,  0],
       [ 1,  1,  0,  0,  0,  0,  1, -1,  1],
       [ 0,  1,  0,  1,  1, -1,  1,  1,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0, -1,  1,  0],
       [ 0,  0, -1,  1,  0,  0,  0,  1,  0],
       [ 0,  0, -1,  1, -1, -1,  1,  0,  0]], dtype=int8)

In [11]:
Qs

{'[[0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]]': -1}

In [12]:
Ns

{'[[0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]]': 1}

In [13]:
for move in b.get_legal_moves(current_color):
    s = copy(b).execute_move(move,current_color).tostring()
    print(s,Qs[s],Ns[s])

KeyError: '[[1 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0 0]]'

In [235]:
b.winner

-1

In [106]:
init_value=(
    np.array([ -1,  14,  26,  -6,  -8,  -5,   2,  18,  12,  20,  -4,  14,  -1,
        -5, -11,   6,  13,   7,   2,  12,   7,  -7,  20,  -5,  17,   1,
         2,   6,  15,  12,  14,   2,  19,  -3,  10,   2,  -1,  -7,  -3,
        14,  22,  24,  -3,  -2,  15, -14,   8,  -7,  16,   9,   1,  -6,
         5, -16,  12,  -6,   9,  -9,  12,  17,  11,  15,   0,   9,  -8,
        15,   4,  10,   9, -10,   5,  -3,   6,   9,   4,  -7,  -2, -11,
        19, -10,  15])+
    np.array([ 88,  97,  76,  89,  47,  43,  95,  27,  64, 118, 108,  56,  53,
        28,  78,  82,  69,  89,  95,  61, 115, -23,  11,  35,  77, 100,
        45,  25,  41, -19, 113,  55, 128,  79,  18,  19,  71,  45, -54,
       103, 123, 123,  23,   1,  49, -14,  -2,  47, 157,  80, 109, -11,
        38,   8, 128,  41,  74,  29,  36,   3, 103, 107,  97,  37,  74,
        55,  22,   8,  80,  53,  70,  88,  60,  87, 118,  50,  51,  -1,
        79,  73,  93])+
    np.array([ 64,  68,  70,  20,  87,  24,  36,  85, 118,  39,  99, 101,  44,
       -35,  19, 105, 106, 135,  88,  69, 122,  39,  63, -14,  83,  72,
        73,  15,  86,  84, 146,  73,  75, -23, 114,  59,  49,  36,  25,
        67, 204,  63,  80,   6,  61,  52, -13, -11, 111, 162,  88,  24,
        31,  20,  45,  14,  28, -22,  10,  88,  66,  47,  -2,  35,  88,
       123,  25,  20,  18,  29,  61,  42, 110,  66,  45,  10,  66,  -6,
       -13,  66,  44])
).reshape(9,9)

In [107]:
init_symmetrized=np.zeros((9,9),dtype=np.int32)
for i in range(1,5):
    init_symmetrized+=np.rot90(init_value,i)
    init_symmetrized+=np.rot90(np.fliplr(init_value),i)

In [109]:
init_symmetrized.reshape(-1).argmax()

40

In [29]:
def one_search(board,color):
    b=copy(board)
    current_color=copy(color)
    while not b.finished:
        move=randchoice(b.get_legal_moves(current_color))
    #         print(move+1)
        b.execute_move(move,current_color)
        current_color*=-1
    return b.winner

In [51]:
np.array([1,2,3]).argmax()

2

In [47]:
import time

time_start=time.time()
print(sum([one_search(b,current_color) for i in range(10000)]))
time_end=time.time()
print('time cost',time_end-time_start,'s')

576
time cost 54.88543772697449 s


In [24]:
b.pieces

array([[ 1,  1, -1,  0,  1, -1, -1,  1,  1],
       [-1, -1,  1,  1, -1, -1,  1, -1,  1],
       [ 1, -1,  1, -1, -1, -1,  1,  1, -1],
       [-1,  0,  1,  1,  0,  1,  0,  1, -1],
       [-1, -1,  1,  0,  0,  1,  1, -1, -1],
       [ 0, -1, -1,  0,  0,  1,  1,  1, -1],
       [-1,  1, -1, -1, -1,  1, -1,  1, -1],
       [-1,  0,  1, -1, -1, -1,  1, -1, -1],
       [-1,  1, -1,  1,  1,  1,  1,  1,  1]], dtype=int8)

In [25]:
b1.pieces

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [10]:
import time

time_start=time.time()
for i in range(100):
    b=Board(9)
    current_color=1
    while not b.finished:
        move=randchoice(b.get_legal_moves(current_color))
#         print(move+1)
        b.execute_move(move,current_color)
        current_color*=-1
    print(b.finished,b.winner)
time_end=time.time()
print('time cost',time_end-time_start,'s')

True 1
True 1
True -1
True -1
True -1
True -1
True 1
True 1
True 0
True -1
True 1
True -1
True -1
True 1
True 1
True 0
True 0
True 1
True -1
True 0
True 1
True 0
True 1
True 1
True -1
True 0
True 1
True 1
True -1
True 1
True 0
True 1
True -1
True 1
True -1
True -1
True -1
True 0
True -1
True -1
True 0
True 1
True 0
True 1
True 1
True 1
True 0
True -1
True 1
True -1
True -1
True -1
True -1
True 1
True 0
True 1
True -1
True 1
True 0
True 0
True -1
True -1
True -1
True 1
True -1
True 0
True 0
True -1
True 1
True 1
True 1
True -1
True 0
True 0
True -1
True 1
True -1
True -1
True 1
True 1
True -1
True 1
True -1
True 1
True -1
True 0
True -1
True 0
True 1
True 1
True -1
True -1
True 1
True -1
True -1
True -1
True 1
True 1
True 0
True -1
time cost 0.5495250225067139 s


In [4]:
(b.finished,b.winner)

(True, -1)

In [5]:
b.main_pieces_finished

array([[1, 1, 1],
       [1, 1, 1],
       [0, 0, 1]], dtype=int8)

In [6]:
b.pieces

array([[ 0,  0, -1,  1,  0, -1,  1,  0,  1],
       [ 0,  1, -1,  0,  1,  0,  1,  1,  1],
       [ 0,  0, -1,  0,  0,  1, -1,  1,  0],
       [-1, -1, -1, -1,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  1, -1,  1,  1,  0, -1],
       [ 1, -1, -1,  1,  0, -1,  1,  0, -1],
       [-1,  0, -1,  1, -1, -1,  1,  0,  0],
       [ 1,  0,  0, -1,  0,  0,  1,  0,  0],
       [-1,  1, -1,  0,  1,  0,  1, -1,  1]], dtype=int8)

In [7]:
b.main_pieces

array([[-1,  1,  1],
       [-1, -1, -1],
       [ 0,  0,  1]], dtype=int8)

In [10]:
randchoice(b.get_legal_moves(current_color))

array([2, 2], dtype=int64)

In [11]:
b.get_legal_moves(current_color)

array([[2, 1],
       [2, 2]], dtype=int64)

In [241]:
import pickle
class test():
    def __init__(self):
        self.value=1
    def tostring(self):
        return pickle.dumps(self)

In [242]:
t=test()

In [245]:
t2=pickle.loads(t.tostring())

In [246]:
t2.value

1